In [1]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

#import warnings
#warnings.filterwarnings('ignore')

(Tutorial_Get)=
# Get
*Retrieving attribute values from a molecular system.*

A molecular system is defined by its attributes and the attributes of its elements — such as the number of atoms, group names, bonded atoms, coordinates, box dimensions, and more. However, not all attributes are available in every representation (form) of a system. Molecular systems can take different forms, such as string identifiers, file paths, or Python objects. Each form includes a specific set of attributes and omits others. Regardless of the form, retrieving attribute values is a key operation in any workflow involving molecular models.

:::{hint}
Visit the section [User guide > Introduction > Molecular System](../../intro/molecular_systems/index.md) in case you are not familiar with the concepts of "form" or "attribute" in MolSysMT.
:::

The list of attributes defined in MolSysMT for a molecular system can be checked in the section [User guide > Introduction > Molecular system > Attributes](../../introduction/molecular_system/attributes.ipynb). And the function {func}`molsysmt.basic.get()` is the tool to get the attribute values of a molecular system in [any supported form](../../introduction/molecular_system/form.ipynb).

## How this function works

:::{admonition} API documentation
Follow this link for a detailed description of the input arguments, raised errors, and returned objects of this function:{func}`molsysmt.basic.get`.
:::

To illustrate how this function works, let's load a molecular system:

In [2]:
import molsysmt as msm

In [3]:
molsys = msm.convert('1TCD', to_form='molsysmt.MolSys')

In [4]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_proteins,n_structures
molsysmt.MolSys,3983,662,167,4,167,2,165,2,1


Let's retrieve the names of atoms with indices 32, 33, and 34 (0-based). The function {func}`molsysmt.basic.get()` uses the element argument to specify the type of element being queried (e.g., 'atom', 'group', 'molecule', etc.). To filter which elements to include, use the selection argument, which accepts a list of indices or a query string.

In [5]:
names = msm.get(molsys, element='atom', selection=[32,33,34], name=True)
print('Atom names:',names)

Atom names: ['N', 'CA', 'C']


:::{tip}
All methods defined in the {ref}`molsysmt.basic <API basic>` module can be invoked also from the main level of the library. Hence, {func}`molsysmt.get` is the same method as {func}`molsysmt.basic.get`.
:::

We are not limited to retrieving just one attribute. Multiple attributes can be requested in a single call:

In [6]:
names, group_indices, group_names = msm.get(molsys, element='atom', selection=[32,33,34],
                                            name=True, group_index=True, group_name=True)
print('Atom names:', names)
print('Group indices:', group_indices)
print('Group names:', group_names)

Atom names: ['N', 'CA', 'C']
Group indices: [4, 4, 4]
Group names: ['ILE', 'ILE', 'ILE']


Let's illustrate now how things can work with other elements:

In [7]:
n_atoms = msm.get(molsys, element='group', selection=[10,11,12], n_atoms=True)
print('Number of atoms:', n_atoms)

Number of atoms: [9, 6, 8]


In [8]:
msm.get(molsys, element='group', selection=[10,11,12], n_atoms=True)

[9, 6, 8]

In [9]:
names, atom_indices, atom_names = msm.get(molsys, element='group', selection=[10,11,12],
                                            name=True, atom_index=True, atom_name=True)
print('Group names:', names)
print('Atom indices:', atom_indices)
print('Atom names:', atom_names)

Group names: ['LYS', 'CYS', 'ASN']
Atom indices: [[77, 78, 79, 80, 81, 82, 83, 84, 85], [86, 87, 88, 89, 90, 91], [92, 93, 94, 95, 96, 97, 98, 99]]
Atom names: [['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'CE', 'NZ'], ['N', 'CA', 'C', 'O', 'CB', 'SG'], ['N', 'CA', 'C', 'O', 'CB', 'CG', 'OD1', 'ND2']]


In [10]:
indices, component_indices = msm.get(molsys, element='component', selection=[55, 56, 57],
                                    index=True, group_index=True)
print('Indices:', indices)
print('Component indices:', component_indices)

Indices: [55, 56, 57]
Component indices: [[550], [551], [552]]


If no `selection` is specified, the function applies to all elements of the chosen type by default. See for example:

In [11]:
msm.get(molsys, element='atom', n_atoms=True)

3983

The function {func}`msm.basic.get()` also supports selection strings using logical or spatial queries. If you're not familiar with selection syntax, visit [User guide > Tools > Basic > Select](select.ipynb). Let's see how ``selection`` works beyond indices lists with some examples:

In [12]:
msm.get(molsys, element='atom', selection='group_index==20', name=True, index=True)

[['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD'], [148, 149, 150, 151, 152, 153, 154]]

In [13]:
msm.get(molsys, element='atom', selection='molecule_type=="protein"', n_groups=True)

497

In [14]:
msm.get(molsys, element='molecule', selection='molecule_type=="water"', n_molecules=True)

165

In [15]:
msm.get(molsys, element='atom', selection='all within 7.0 angstroms of atom_index==0', n_atoms=True)

22

Finally, not all molecular system forms include all attributes. For example, an `openmm.Topology` object has no atom coordinates, and a `file:inpcrd` system may not contain chain names. When an attribute is unavailable, the function returns `None`.

In [16]:
inpcrd_file = msm.systems['pentalanine']['pentalanine.inpcrd']
msm.get(inpcrd_file, n_groups=True) is None

True

## Examples with topological attributes

Here you can find some additional examples where {func}`msm.basic.get()` works over atoms, groups, components, chains, molecules, entities, bonds or system.

### With atoms

In [17]:
# Group id of atoms with index 0, 1 or 2
msm.get(molsys, element='atom', selection='atom_index in [0,1,2]', group_id=True)

[4, 4, 4]

In [18]:
# Number of groups in atoms with index 0, 1 or 2
msm.get(molsys, element='atom', selection='atom_index in [0,1,2]', n_groups=True)

1

In [19]:
# Index of groups in atoms with index 0 to 15
msm.get(molsys, element='atom', selection=range(5,10), group_index=True)

[0, 0, 0, 0, 1]

### With groups

In [20]:
# Id of groups with index 0, 1 or 2
msm.get(molsys, element='group', selection=[0,1,2], id=True)

[4, 5, 6]

In [21]:
# Index of atoms in groups with index 0 or 1
msm.get(molsys, element='group', selection=[0,1], atom_index=True)

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15]]

In [22]:
# Number of groups in molecules of type water
msm.get(molsys, element='group', selection='molecule_type=="water"', n_groups=True)

165

In [23]:
# Number of groups of type aminoacid
msm.get(molsys, element='group', selection='group_type=="amino acid"', n_groups=True)

497

### With components

In [24]:
msm.get(molsys, element='component', selection='molecule_type=="water"', index=True)

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166]

In [25]:
msm.get(molsys, element='component', selection=[0,1,2,3], n_waters=True)

2

### With molecules

In [26]:
# Names of groups in molecule of index 0
msm.get(molsys, element='molecule', selection=[0,1], group_name=True)

[['LYS',
  'PRO',
  'GLN',
  'PRO',
  'ILE',
  'ALA',
  'ALA',
  'ALA',
  'ASN',
  'TRP',
  'LYS',
  'CYS',
  'ASN',
  'GLY',
  'SER',
  'GLU',
  'SER',
  'LEU',
  'LEU',
  'VAL',
  'PRO',
  'LEU',
  'ILE',
  'GLU',
  'THR',
  'LEU',
  'ASN',
  'ALA',
  'ALA',
  'THR',
  'PHE',
  'ASP',
  'HIS',
  'ASP',
  'VAL',
  'GLN',
  'CYS',
  'VAL',
  'VAL',
  'ALA',
  'PRO',
  'THR',
  'PHE',
  'LEU',
  'HIS',
  'ILE',
  'PRO',
  'MET',
  'THR',
  'LYS',
  'ALA',
  'ARG',
  'LEU',
  'THR',
  'ASN',
  'PRO',
  'LYS',
  'PHE',
  'GLN',
  'ILE',
  'ALA',
  'ALA',
  'GLN',
  'ASN',
  'ALA',
  'ILE',
  'THR',
  'ARG',
  'SER',
  'GLY',
  'ALA',
  'PHE',
  'THR',
  'GLY',
  'GLU',
  'VAL',
  'SER',
  'LEU',
  'GLN',
  'ILE',
  'LEU',
  'LYS',
  'ASP',
  'TYR',
  'GLY',
  'ILE',
  'SER',
  'TRP',
  'VAL',
  'VAL',
  'LEU',
  'GLY',
  'HIS',
  'SER',
  'GLU',
  'ARG',
  'ARG',
  'LEU',
  'TYR',
  'TYR',
  'GLY',
  'GLU',
  'THR',
  'ASN',
  'GLU',
  'ILE',
  'VAL',
  'ALA',
  'GLU',
  'LYS',
  'VAL',
 

In [27]:
# Number of molecules of type protein
msm.get(molsys, element='molecule', selection='molecule_type=="protein"', n_molecules=True)

2

In [28]:
# Molecule type of molecules with index 1 to 9
msm.get(molsys, element='molecule', selection=range(1,10), molecule_type=True)

['protein',
 'water',
 'water',
 'water',
 'water',
 'water',
 'water',
 'water',
 'water']

### With chains

In [29]:
# Molecule type of molecules with index 1 to 9
msm.get(molsys, element='chain', selection='all', name=True)

['A', 'B', 'A', 'B']

In [30]:
# Molecule type of molecules with index 1 to 9
msm.get(molsys, element='chain', selection='all', id=True)

[0, 1, 2, 3]

In [31]:
# Molecule type of molecules with index 1 to 9
msm.get(molsys, element='chain', selection='molecule_type=="protein"', id=True)

[0, 1]

### With entities

In [32]:
# Name of entity with index 0
msm.get(molsys, element='entity', selection=0, name=True)

['TRIOSEPHOSPHATE ISOMERASE']

In [33]:
# Number of molecules in entity with index 1
msm.get(molsys, element='entity', selection=1, n_molecules=True)

[165]

In [34]:
# Index of molecules in entity with index 1
msm.get(molsys, element='entity', selection=1, molecule_index=True)

[[2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,


### With bonds

In [35]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], bonded_atoms=True)

[[1], [0, 2, 4], [1, 3, 9], [2], [1, 5], [4, 6]]

In [36]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], inner_bonded_atoms=True)

[[1], [0, 2, 4], [1, 3], [2], [1, 5], [4]]

In [37]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], bonded_atoms_pairs=True)

[[0, 1], [1, 2], [1, 4], [2, 3], [2, 9], [4, 5], [5, 6]]

In [38]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], inner_bonded_atoms_pairs=True)

[[0, 1], [1, 2], [1, 4], [2, 3], [4, 5]]

In [39]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], bond_index=True)

[[0], [0, 1, 2], [1, 3, 4], [3], [2, 5], [5, 6]]

In [40]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], n_bonds=True)

7

In [41]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], inner_bond_index=True)

[[0], [0, 1, 2], [1, 3], [3], [2, 5], [5]]

In [42]:
msm.get(molsys, element='atom', selection=[0,1,2,3,4,5], n_inner_bonds=True)

5

In [43]:
msm.select(molsys, element='bond', selection='group_index==3')

[26, 27, 28, 29, 30, 32, 33]

In [44]:
msm.get(molsys, element='bond', selection='group_index==3', index=True)

[26, 27, 28, 29, 30, 32, 33]

In [45]:
msm.get(molsys, element='bond', selection=[0,1,2,3,4], bonded_atoms=True)

[[0, 1], [1, 2], [1, 4], [2, 3], [2, 9]]

In [46]:
msm.get(molsys, element='bond', selection='group_index==3', bonded_atoms=True)

[[25, 26], [25, 31], [26, 27], [26, 29], [27, 28], [29, 30], [30, 31]]

### With the system

The element system has some specific observables accounting for the amount of type of elements such as the number of aminoacid groups, ion molecules or protein molecules:

In [47]:
# Number of aminoacids and number of waters in the system
msm.get(molsys, element='system', n_amino_acids=True, n_waters=True)

[497, 165]

In [48]:
msm.get(molsys, element='system', n_bonds=True)

3890

## Examples with structural attributes

In [49]:
msm.get(molsys, element='system', n_structures=True)

1

In [50]:
msm.get(molsys, element='atom', selection=100, structure_indices=0, coordinates=True)

Magnitude,[[[1.8835 3.8270999999999997 5.0365]]]
Units,nanometer


In [51]:
msm.get(molsys, element='system', coordinates=True)

Magnitude,[[[1.8466999999999998 2.7638 8.3661] [1.8580999999999999 2.8457 8.2415] [1.7231 2.8536999999999995 8.1751] ... [2.685 6.2692 5.3171] [0.0135 4.6709 3.6203] [0.5324 6.512399999999999 5.142499999999999]]]
Units,nanometer


In [52]:
msm.get(molsys, element='system', structure_indices=0, box=True)

Magnitude,[[[4.3709999999999996 0.0 0.0] [0.0 7.765 0.0] [0.0 0.0 14.953999999999997]]]
Units,nanometer


In [53]:
box_lengths, box_angles, box_volume = msm.get(molsys, element='system', structure_indices=0,
                                              box_lengths=True, box_angles=True, box_volume=True)

In [54]:
box_lengths

Magnitude,[[4.371 7.765 14.954]]
Units,nanometer


In [55]:
box_angles

Magnitude,[[89.99998127603166 89.99998127603166 89.99998127603166]]
Units,degree


In [56]:
box_volume

Magnitude,[507.55094750999984]
Units,nanometer3


:::{seealso}
[User guide > Introduction > Molecular System > Forms](../../intro/molecular_systems/forms.md):    
Describe the different forms (representations) used for molecular systems.

[User guide > Introduction > Molecular System > Attributes](../../intro/molecular_systems/attributes.ipynb):    
List and define the standard attributes available in MolSysMT.

[User guide > Tools > Basic > Convert](convert.ipynb):    
Convert a molecular system into another form.

[User guide > Tools > Basic > Info](info.ipynb):    
Display a summary of the contents and structure of a molecular system.

[User guide > Tools > Basic > Select](selection.ipynb):    
Select specific elements from a molecular system using index-based or query-based selection.

[User guide > Tools > Basic > Get attributes](get_attributes.ipynb):    
Check which attributes are present in a molecular system.

[User guide > Tools > Basic > Has attribute](has_attribute.ipynb):    
Determine whether a molecular system includes a specific attribute.

:::